In [1]:
print('Hello world')

Hello world


In [1]:
import torch

/opt/miniconda3/envs/test_codes/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


## Vanilla Self Attention:

### The inputs are given below

In [2]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
query = inputs[0]
print(query)

tensor([0.4300, 0.1500, 0.8900])


### Calculating attention vectors

In [4]:
dp = {}
def calculate_dot_product(i, inputs):
    dot_products = []
    for j in range(len(inputs)):
        dot_products.append(torch.dot(inputs[i], inputs[j]))
    return dot_products
for i in range(len(inputs)):
    dp[i] = calculate_dot_product(i, inputs)
print(dp)

{0: [tensor(0.9995), tensor(0.9544), tensor(0.9422), tensor(0.4753), tensor(0.4576), tensor(0.6310)], 1: [tensor(0.9544), tensor(1.4950), tensor(1.4754), tensor(0.8434), tensor(0.7070), tensor(1.0865)], 2: [tensor(0.9422), tensor(1.4754), tensor(1.4570), tensor(0.8296), tensor(0.7154), tensor(1.0605)], 3: [tensor(0.4753), tensor(0.8434), tensor(0.8296), tensor(0.4937), tensor(0.3474), tensor(0.6565)], 4: [tensor(0.4576), tensor(0.7070), tensor(0.7154), tensor(0.3474), tensor(0.6654), tensor(0.2935)], 5: [tensor(0.6310), tensor(1.0865), tensor(1.0605), tensor(0.6565), tensor(0.2935), tensor(0.9450)]}


### Calculating normalized attention vectors

In [8]:
def torch_softmax(x):
    return torch.exp(x) / torch.sum(torch.exp(x), dim=0)
for element in dp:
    dp[element] = torch_softmax(torch.tensor(dp[element]))
print(dp)

{0: tensor([0.1893, 0.1849, 0.1837, 0.1438, 0.1424, 0.1560]), 1: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581]), 2: tensor([0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565]), 3: tensor([0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720]), 4: tensor([0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295]), 5: tensor([0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896])}


### Calculating context vectors:

In [14]:
def calculate_context_vector(dp_element, inputs):
    context_vector = torch.matmul(dp_element,inputs)
    return context_vector
for element in dp:
    dp[element] = calculate_context_vector(dp[element], inputs)
print(dp)

{0: tensor([0.4369, 0.5891, 0.5555]), 1: tensor([0.4419, 0.6515, 0.5683]), 2: tensor([0.4431, 0.6496, 0.5671]), 3: tensor([0.4304, 0.6298, 0.5510]), 4: tensor([0.4671, 0.5910, 0.5266]), 5: tensor([0.4177, 0.6503, 0.5645])}


### In Matrix form 

In [16]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):

      attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [19]:
attn_socres = inputs@inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [21]:
attn_weights = torch.softmax(attn_scores, dim=-1)
# dim = -1 means normalize across the columns so that sum of each row is 1
print(attn_weights)
print(attn_weights.sum(dim=-1))

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [22]:
context_vectors = attn_weights@inputs
print(context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [23]:
def self_attention(inputs):
    attn_scores = inputs@inputs.T
    attn_weights = torch.softmax(attn_scores, dim = -1)
    context_vectors = attn_weights@inputs
    return context_vectors
print(self_attention(inputs))

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self attention as in Transformer architecture

In [24]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [26]:
x1 = inputs[0]
d_in = inputs.shape[1]
d_out = 2

In [27]:
torch.manual_seed(0)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)


In [29]:
W_query

Parameter containing:
tensor([[0.4963, 0.7682],
        [0.0885, 0.1320],
        [0.3074, 0.6341]])

In [31]:
# if we want to calculate attention weigths for x1, then also we need keys and values for all the inputs as they are involved in the calculation of attention weights for x1
query = x1@W_query
keys = inputs@W_key
values= inputs@W_value
print(query)

tensor([0.5003, 0.9145])


In [35]:
keys

tensor([[0.5896, 0.8378],
        [0.8962, 1.3083],
        [0.8899, 1.3055],
        [0.4872, 0.6965],
        [0.5262, 0.8885],
        [0.5809, 0.7716]])

In [34]:
keys_1 = keys[0]
attn_scores_1 = query.dot(keys_1)
print(attn_scores_1)

tensor(1.0611)


In [37]:
attn_scores_1 = query@keys.T
print(attn_scores_1)

tensor([1.0611, 1.6447, 1.6391, 0.8807, 1.0757, 0.9962])


In [38]:
d_k = keys.shape[-1]
attn_weights_1 = torch.softmax(attn_scores_1/ d_k**0.5, dim = -1)
print(attn_weights_1)

tensor([0.1457, 0.2202, 0.2193, 0.1283, 0.1473, 0.1392])


In [40]:
context_vectors_1 = attn_weights_1@values
print(context_vectors_1)

tensor([0.5762, 0.8392])


### Self attention class (v1 -> without masking)

In [44]:
import torch.nn as nn
class SelfAttention(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
        self.W_key = nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
        self.W_value = nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
    def forward(self, x):
        keys = x@self.W_key
        values = x@self.W_value
        query = x@self.W_query
        attn_scores = query@keys.T
        d_k = keys.shape[-1]
        attn_weights = torch.softmax(attn_scores/d_k**0.5, dim = -1)
        context_vec = attn_weights@values
        return context_vec
    

In [45]:
torch.manual_seed(123)
attn_vec = SelfAttention(d_in, d_out)
print(attn_vec(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


Using nn.Linear Layers -> nn.Linear has an optimized weight initialization scheme which is more stable and effective in training.

In [46]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
 
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [48]:
torch.manual_seed(123)
attn_vec = SelfAttention_v2(d_in, d_out)
print(attn_vec(inputs))

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)


### Masked Self Attention

In [54]:
query = attn_vec.W_query(inputs)
keys = attn_vec.W_key(inputs)
values = attn_vec.W_value(inputs)
attn_scores = query@keys.T
attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim = -1)
print(attn_weights)

tensor([[0.1717, 0.1762, 0.1761, 0.1555, 0.1627, 0.1579],
        [0.1636, 0.1749, 0.1746, 0.1612, 0.1605, 0.1652],
        [0.1637, 0.1749, 0.1746, 0.1611, 0.1606, 0.1651],
        [0.1636, 0.1704, 0.1702, 0.1652, 0.1632, 0.1674],
        [0.1667, 0.1722, 0.1721, 0.1618, 0.1633, 0.1639],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682]],
       grad_fn=<SoftmaxBackward0>)


In [55]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [57]:
masked_simple = attn_weights*mask_simple


In [60]:
row_sums = masked_simple.sum(dim = 1, keepdim = True)
masked_simple_norm = masked_simple/row_sums


mask with -infinty

In [74]:
mask_simple = torch.triu(torch.ones(context_length, context_length), diagonal = 1)
masked = attn_scores.masked_fill(mask_simple.bool(), -torch.inf)
print(mask_simple)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1667, 0.2602, 0.2577,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1064, 0.0643,   -inf,   -inf],
        [0.1415, 0.1875, 0.1863, 0.0987, 0.1121,   -inf],
        [0.0476, 0.1192, 0.1171, 0.0731, 0.0477, 0.0966]],
       grad_fn=<MaskedFillBackward0>)


In [75]:
attn_weights = torch.softmax(masked/keys.shape[0]**0.5, dim = -1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4903, 0.5097, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3250, 0.3377, 0.3373, 0.0000, 0.0000, 0.0000],
        [0.2468, 0.2526, 0.2524, 0.2481, 0.0000, 0.0000],
        [0.1997, 0.2035, 0.2034, 0.1962, 0.1973, 0.0000],
        [0.1642, 0.1691, 0.1690, 0.1659, 0.1642, 0.1675]],
       grad_fn=<SoftmaxBackward0>)


In [76]:
context_vec = attn_weights@values
print(context_vec)

tensor([[-0.4519,  0.2216],
        [-0.5856,  0.0087],
        [-0.6284, -0.0607],
        [-0.5664, -0.0832],
        [-0.5511, -0.0978],
        [-0.5290, -0.1073]], grad_fn=<MmBackward0>)


mask with dropout

In [78]:
torch.manual_seed(123)
dropout = nn.Dropout(0.5)
print(torch.ones(6,6),dropout(torch.ones(6,6)))

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]]) tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [83]:
dropout_attn_weights = dropout(attn_weights)
print(dropout_attn_weights)

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0193, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6500, 0.6753, 0.6746, 0.0000, 0.0000, 0.0000],
        [0.4936, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3994, 0.4069, 0.4067, 0.0000, 0.3946, 0.0000],
        [0.0000, 0.0000, 0.3379, 0.0000, 0.0000, 0.3351]],
       grad_fn=<MulBackward0>)


In [84]:
context_vec = dropout_attn_weights@values
print(context_vec)

tensor([[ 0.0000,  0.0000],
        [-0.7280, -0.1999],
        [-1.2569, -0.1213],
        [-0.2231,  0.1094],
        [-0.9528, -0.1345],
        [-0.3820, -0.1169]], grad_fn=<MmBackward0>)


### Causal Attention class

In [86]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal = 1))
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1,2)
        attn_scores.masked_fill(self.mask.bool()[:num_tokens,:num_tokens], -torch.inf)

        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim = -1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights@values
        return context_vec



In [88]:
torch.manual_seed(123)
batch = torch.stack((inputs, inputs), dim = 0)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vec = ca(batch)
context_vec, context_vec.shape

(tensor([[[-0.5337, -0.1051],
          [-0.5323, -0.1080],
          [-0.5323, -0.1079],
          [-0.5297, -0.1076],
          [-0.5311, -0.1066],
          [-0.5299, -0.1081]],
 
         [[-0.5337, -0.1051],
          [-0.5323, -0.1080],
          [-0.5323, -0.1079],
          [-0.5297, -0.1076],
          [-0.5311, -0.1066],
          [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>),
 torch.Size([2, 6, 2]))

## Multi-head Attention:

In [92]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim = -1)


In [93]:
torch.manual_seed(123)
context_length = inputs.shape[0]
d_in, d_out = 3,2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads = 2)
context_vecs = mha(batch)

In [94]:
print(context_vecs, context_vecs.shape)

tensor([[[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>) torch.Size([2, 6, 4])
